<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/Fragility_Index_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
# Fragility metrics for a 2×2 table, Google Colab ready
# Walsh-compliant FI + FQ + MFQ  ➕  Global Fragility Index (GFI) + Global Fragility Quotient (GFQ)
#
# Layout:
#   |  Events | Non-events |
# A |    a    |     b      |
# B |    c    |     d      |
#
# Tests: Fisher's exact (two-sided), α = 0.05.
#
# FI (Walsh rules): toggles only within the arm with FEWER events at baseline;
# if tied, choose arm with FEWER total; if still tied, choose A.
# A toggle is an event↔non-event flip within that arm. Row totals fixed.
#
# FQ  = FI / N
# MFQ = (fragility count) / n_mod   (n_mod = size of the modified arm at baseline; by default fragility count = FI)
#
# NEW (Global, path-independent under fixed N):
# GFI = minimum number of single-observation moves between ANY two cells (N fixed)
#       required to flip significance (two-sided Fisher).
# GFQ = GFI / N
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit (Version 2.1.0) [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

# Install SciPy if missing
try:
    import scipy
except ImportError:
    # In Colab this will work; in other environments use pip accordingly
    !pip -q install scipy

from collections import deque
from scipy.stats import fisher_exact

ALPHA = 0.05

# ---------- Core utilities ----------

def fisher_p(a, b, c, d):
    _, p = fisher_exact([[a, b], [c, d]], alternative="two-sided")
    return p

def is_significant(p, alpha=ALPHA):
    return p <= alpha

def n_total(a, b, c, d):
    return a + b + c + d

# ---------- Walsh-compliant FI / FQ / MFQ ----------

def choose_arm(a, b, c, d):
    # Arm with fewer events; if tie, arm with fewer total; if still tie, A
    if a < c:
        return 'A'
    if c < a:
        return 'B'
    totA, totB = a + b, c + d
    if totA < totB:
        return 'A'
    if totB < totA:
        return 'B'
    return 'A'

def toggle_once(a, b, c, d, arm, direction):
    """
    Apply one within-arm toggle.
    direction ∈ {"up","down"}:
      up   = non-event -> event  (increase events in chosen arm)
      down = event     -> non-event (decrease events in chosen arm)
    Returns new counts or None if impossible.
    """
    if arm == 'A':
        if direction == 'up':
            if b <= 0: return None
            return a+1, b-1, c, d
        else:  # down
            if a <= 0: return None
            return a-1, b+1, c, d
    else:  # 'B'
        if direction == 'up':
            if d <= 0: return None
            return a, b, c+1, d-1
        else:  # down
            if c <= 0: return None
            return a, b, c-1, d+1

def steps_to_cross(a, b, c, d, arm, direction, alpha=ALPHA, max_iter=10**6):
    """
    Apply monotone toggles within the chosen arm until significance flips.
    Returns (steps, final_p) or (None, last_p) if impossible.
    """
    base_p = fisher_p(a, b, c, d)
    base_sig = is_significant(base_p, alpha)

    steps = 0
    A,B,C,D = a,b,c,d
    for _ in range(max_iter):
        nxt = toggle_once(A,B,C,D, arm, direction)
        if nxt is None:
            return None, fisher_p(A,B,C,D)
        A,B,C,D = nxt
        steps += 1
        p_now = fisher_p(A,B,C,D)
        if is_significant(p_now, alpha) != base_sig:
            return steps, p_now
    return None, fisher_p(A,B,C,D)

def compute_fi_fq_mfq(a, b, c, d, alpha=ALPHA):
    base_p = fisher_p(a,b,c,d)
    base_state = "significant" if is_significant(base_p, alpha) else "non-significant"
    arm = choose_arm(a,b,c,d)

    up_res   = steps_to_cross(a,b,c,d, arm, "up",   alpha)
    down_res = steps_to_cross(a,b,c,d, arm, "down", alpha)

    candidates = []
    if up_res[0]   is not None: candidates.append(("non-events → events", up_res[0], up_res[1]))
    if down_res[0] is not None: candidates.append(("events → non-events", down_res[0], down_res[1]))

    N = n_total(a,b,c,d)
    n_mod = (a + b) if arm == 'A' else (c + d)

    if not candidates:
        return {
            "FI": None, "FQ": None, "MFQ": None,
            "baseline_p": base_p, "baseline_state": base_state,
            "arm": arm, "direction": None, "n_mod": n_mod,
            "final_p": base_p, "note": "FI not attainable under Walsh constraints."
        }

    direction, steps, p_final = min(candidates, key=lambda x: x[1])
    FI = int(steps)
    FQ = (FI / N) if N > 0 else None
    MFQ = (FI / n_mod) if n_mod > 0 else None

    return {
        "FI": FI, "FQ": FQ, "MFQ": MFQ,
        "baseline_p": base_p, "baseline_state": base_state,
        "target_state": ("non-significant" if base_state=="significant" else "significant"),
        "arm": arm, "direction": direction, "n_mod": n_mod, "final_p": p_final
    }

# ---------- Global Fragility Index (GFI) / GFQ for 2×2 ----------

# State index mapping: 0:a, 1:b, 2:c, 3:d
def _neighbors_all_moves(state):
    """Generate all one-move neighbors: decrement one cell, increment another (nonnegative constraint)."""
    s = list(state)
    for src in range(4):
        if s[src] == 0:
            continue
        for dst in range(4):
            if dst == src:
                continue
            t = s.copy()
            t[src] -= 1
            t[dst] += 1
            yield tuple(t), (src, dst)

def compute_gfi_gfq(a, b, c, d, alpha=ALPHA, depth_cap=None):
    """
    Global, path-independent minimal moves (GFI) to flip two-sided Fisher significance.
    Searches the multinomial lattice with total N fixed.
    Returns dict with GFI, GFQ, final p, and witness path of moves (indices 0..3).
    """
    # Validate
    for x in (a,b,c,d):
        if not (isinstance(x, int) and x >= 0):
            raise ValueError("Cells a,b,c,d must be non-negative integers.")

    N = n_total(a,b,c,d)
    if N == 0:
        return {"GFI": None, "GFQ": None, "baseline_p": None, "baseline_state": None,
                "final_p": None, "witness_path": [], "note": "Empty table."}

    base_state = is_significant(fisher_p(a,b,c,d), alpha)
    start = (a,b,c,d)

    # Reasonable depth cap to avoid pathological loops; minimal flip will not exceed N for 2×2 in practice
    if depth_cap is None:
        depth_cap = N  # can increase if needed

    visited = {start}
    q = deque()
    q.append( (start, 0, []) )

    while q:
        state, depth, path = q.popleft()
        if depth >= depth_cap:
            continue
        for nxt, move in _neighbors_all_moves(state):
            if nxt in visited:
                continue
            visited.add(nxt)
            new_depth = depth + 1
            p_now = fisher_p(*nxt)
            if is_significant(p_now, alpha) != base_state:
                GFI = new_depth
                GFQ = GFI / N
                return {
                    "GFI": GFI, "GFQ": GFQ,
                    "baseline_p": fisher_p(*start), "baseline_state": ("significant" if base_state else "non-significant"),
                    "final_p": p_now, "witness_path": path + [move]
                }
            q.append( (nxt, new_depth, path + [move]) )

    # If not found within depth_cap, report not attained
    return {
        "GFI": None, "GFQ": None,
        "baseline_p": fisher_p(*start), "baseline_state": ("significant" if base_state else "non-significant"),
        "final_p": fisher_p(*start), "witness_path": [],
        "note": f"No flip found within depth_cap={depth_cap} moves."
    }

# ---------- CLI ----------

def main():
    print("Enter 2×2 table cells as integers.")
    a = int(input("a (Arm A events): ").strip())
    b = int(input("b (Arm A non-events): ").strip())
    c = int(input("c (Arm B events): ").strip())
    d = int(input("d (Arm B non-events): ").strip())

    # FI / FQ / MFQ (Walsh)
    res_fi = compute_fi_fq_mfq(a,b,c,d, alpha=ALPHA)

    # GFI / GFQ (global, multinomial moves)
    res_gfi = compute_gfi_gfq(a,b,c,d, alpha=ALPHA)

    print("\n--- Baseline ---")
    print(f"Fisher p = {fisher_p(a,b,c,d):.6f} (α={ALPHA})")

    print("\n--- Walsh-compliant Fragility ---")
    if res_fi["FI"] is None:
        print("FI: not attainable under Walsh constraints.")
    else:
        print(f"FI  = {res_fi['FI']}")
        print(f"FQ  = {res_fi['FQ']:.6f}")
        print(f"MFQ = {res_fi['MFQ']:.6f}")
        print(f"Chosen arm = {res_fi['arm']}, direction = {res_fi['direction']}, n_mod = {res_fi['n_mod']}")
        print(f"Final p after FI toggles = {res_fi['final_p']:.6f}")

    print("\n--- Global Fragility (path-independent, N fixed) ---")
    if res_gfi["GFI"] is None:
        print(f"GFI: not attained within depth cap. {res_gfi.get('note','')}")
    else:
        print(f"GFI = {res_gfi['GFI']}")
        print(f"GFQ = {res_gfi['GFQ']:.6f}")
        print(f"Final p after GFI moves = {res_gfi['final_p']:.6f}")
        # Optional: show witness path in human terms
        idx_to_name = {0:"a",1:"b",2:"c",3:"d"}
        path_str = " → ".join([f"{idx_to_name[i]}→{idx_to_name[j]}" for (i,j) in res_gfi["witness_path"]])
        print(f"Witness path (cell moves): {path_str if path_str else '(single move)'}")

if __name__ == "__main__":
    main()


Enter 2×2 table cells as integers.
a (Arm A events): 19
b (Arm A non-events): 8
c (Arm B events): 6
d (Arm B non-events): 19

--- Baseline ---
Fisher p = 0.001050 (α=0.05)

--- Walsh-compliant Fragility ---
FI  = 5
FQ  = 0.096154
MFQ = 0.200000
Chosen arm = B, direction = non-events → events, n_mod = 25
Final p after FI toggles = 0.091219

--- Global Fragility (path-independent, N fixed) ---
GFI = 4
GFQ = 0.076923
Final p after GFI moves = 0.050472
Witness path (cell moves): a→b → a→b → a→c → d→c
